# Easy Planning

In [1]:
import pandas as pd

df = pd.read_csv("csv" + "/" + "easy_planning.csv", sep = "\t")

# Riepilogo

In [2]:
def durata(inizio, fine):
    inizio = inizio.replace(":", ".")
    inizio = inizio.replace(".15", ".25")
    inizio = inizio.replace(".30", ".50")
    inizio = inizio.replace(".45", ".70")
    fine = fine.replace(":", ".")
    fine = fine.replace(".15", ".25")
    fine = fine.replace(".30", ".50")
    fine = fine.replace(".45", ".70")
    return float(fine) - float(inizio)

def media_presenze(df, biblioteca, anno, settimana):
    dfTMP = df[(df["biblioteca"] == biblioteca) & (df["anno"] == anno) & (df["settimana"] == settimana) &
        (df["Stato gestione"] == "evaso")]
    s = dfTMP.groupby(["giorno", "intervallo"]).size()
    if len(s) == 0:
        return 0
    return s.mean()

def picco_presenze(df, biblioteca, anno, settimana):
    dfTMP = df[(df["biblioteca"] == biblioteca) & (df["anno"] == anno) & (df["settimana"] == settimana) &
        (df["Stato gestione"] == "evaso")]
    s = dfTMP.groupby(["giorno", "intervallo"]).size()
    if len(s) == 0:
        return 0
    return s.max()

def elenco_presenze(df):
    elenco_presenze = ""
    s = df[(df["Stato gestione"] == "evaso")].groupby(["giorno", "intervallo"]).size()
    dfTMP = s.to_frame(name = "presenze")
    dfTMP.reset_index(inplace = True)
    dfTMP.sort_values(by = ["giorno", "intervallo"], inplace = True)
    for index, row in dfTMP.iterrows():
        elenco_presenze = elenco_presenze + row["giorno"] + ", " + row["intervallo"] + ": " + str(row["presenze"]) + " * "
    return elenco_presenze

with open('csv' + "/" + "easy_planning_riepilogo.csv", "w") as file_handler:

    file_handler.write("Biblioteca" + "\t") 
    file_handler.write("Settimana" + "\t")
    file_handler.write("Inizio" + "\t")
    file_handler.write("Fine" + "\t")
    file_handler.write("Giorni di apertura" + "\t")
    file_handler.write("Posti a sedere" + "\t")
    file_handler.write("Apertura" + "\t")
    file_handler.write("Chiusura" + "\t")
    file_handler.write("Ore di apertura" + "\t")
    file_handler.write("Intervalli" + "\t")
    file_handler.write("Slot disponibili" + "\t")
    file_handler.write("Slot occupati" + "\t")
    file_handler.write("Ore disponibili" + "\t")
    file_handler.write("Ore occupate" + "\t")
    file_handler.write("Presenze" + "\t")
    file_handler.write("Elenco presenze" + "\t")
    file_handler.write("Media" + "\t")
    file_handler.write("Picco")
    file_handler.write("\n")
    
    for anno in df["anno"].unique():
        for settimana in df["settimana"].unique():
            for biblioteca in df["biblioteca"].unique():
                dfTMP = df[(df["biblioteca"] == biblioteca) & (df["anno"] == anno) & (df["settimana"] == settimana)]
                if dfTMP.shape[0] != 0:
                    file_handler.write(biblioteca + "\t") 
                    file_handler.write(str(settimana) + "\t")
                    file_handler.write(str(dfTMP["giorno"].min()) + "\t")
                    file_handler.write(str(dfTMP["giorno"].max()) + "\t")
                    giorni_di_apertura = dfTMP.groupby(["giorno"]).size().shape[0]
                    file_handler.write(str(giorni_di_apertura) + "\t")
                    posti_a_sedere = dfTMP["posti_a_sedere"].mean()
                    file_handler.write(str(posti_a_sedere) + "\t")
                    apertura = dfTMP["intervallo"].min()[0:5]
                    file_handler.write(apertura + "\t")
                    chiusura = dfTMP["intervallo"].max()[6:]                
                    file_handler.write(chiusura + "\t")
                    file_handler.write(str(durata(apertura, chiusura)) + "\t")
                    intervalli = dfTMP.groupby(["intervallo"]).size().shape[0]
                    file_handler.write(str(intervalli) + "\t")
                    file_handler.write(str(giorni_di_apertura * posti_a_sedere * intervalli) + "\t")
                    posti_occupati = dfTMP[(dfTMP["Stato gestione"] == "evaso")].groupby(["settimana"]).size()
                    file_handler.write(str(posti_occupati.sum()) + "\t")
                    dfTMP[(dfTMP["Stato gestione"] == "evaso")]["ore"].sum()
                    file_handler.write(str(giorni_di_apertura * posti_a_sedere * durata(apertura, chiusura)) + "\t")
                    # file_handler.write(str(ore_intervalli(dfTMP)) + "\t") 
                    file_handler.write(str(dfTMP[(dfTMP["Stato gestione"] == "evaso")]["ore"].sum()) + "\t")
                    presenze = dfTMP[(dfTMP["Stato gestione"] == "evaso")].groupby(["Utente", "giorno", "intervallo"]).size().shape[0]
                    file_handler.write(str(presenze) + "\t")
                    file_handler.write(elenco_presenze(dfTMP) + "\t")
                    file_handler.write(str(media_presenze(df, biblioteca, anno, settimana)) + "\t")
                    file_handler.write(str(picco_presenze(df, biblioteca, anno, settimana)))
                    file_handler.write("\n")

In [3]:
dfTMP = pd.read_csv("csv" + "/" + "easy_planning_riepilogo.csv", sep = "\t")

dfTMP.to_excel("xlsx" + "/" + "easy_planning_riepilogo.xlsx")